# Object-Oriented Demo

This is a demo for the class-based features of `climpred` that are in development.

Author: Riley X. Brady (riley.brady@colorado.edu)

Date Last Updated: May 17th, 2019

**Object Types**

There are two types of objects that can be used in `climpred`, and both are built on top of `xarray`'s `Dataset` and `DataArray` objects.

1. `HindcastEnsemble`: This is a climate prediction ensemble that is initialized from some observation-based product. This could be observations, an assimilative product, a reconstruction, etc. The point is that this ensemble is making retrospective forecasts (hindcasts) of the real world.
1. `PerfectModelEnsemble`: This is a climate prediction ensemble that is initialized from a control run, using the "perfect model" framework.

Both the `HindcastEnsemble` and `PerfectModelEnsemble` are sub-classes of a `PredictionEnsemble`.

**Functions**

Functions can be called directly on the `HindcastEnsemble` or `PerfectModelEnsemble` object. `climpred` manages the nitty-gritty of calling the appropriate functions and applying statistics to multiple simulations/observations and variables.

**HindcastEnsemble Functions**

These are the current functions that are supported for `HindcastEnsemble`s. More will be added in the future.

1. `add_reference`: Add a reference simulation/observation to compare the initialized ensemble to.
1. `add_uninitialized`: Add an uninitialized companion ensemble.
1. `compute_metric`: Compare the initialized ensemble to references, e.g., pearson correlation, mean absolute error, root mean square error.
1. `compute_uninitialized`: Compare the uninitialized ensemble to references.
1. `compute_persistence`: Compute a persistence forecast on a reference.


**PerfectModelEnsemble Functions**

These are the current functions that are supported for `PerfectModelEnsemble`s. More will be added in the future.

1. `add_control`: Add the control run that initialized the climate prediction ensemble.
1. `generate_uninitialized`: Generate an uninitialized ensemble by bootstrapping the initialized ensemble.
1. `compute_metric`: Compare the initialized ensemble to the control run.
1. `compute_persistence`: Compute a persistence forecast for the control run.
1. `compute_uninitialized`: Compare the uninitialized ensemble to the control.
1. `bootstrap`: Bootstrap the ensemble simulations with replacement.

In [ ]:
import numpy as np
import xarray as xr
import climpred as cp
import matplotlib.pyplot as plt
%matplotlib inline
from climpred.tutorial import load_dataset

In [ ]:
load_dataset()

## HindcastEnsemble

Here I will use the Community Earth System Model Decadal Prediction Large Ensemble (CESM-DPLE) for a demonstration of the `HindcastEnsemble` capabilities. CESM-DPLE is initialized on November 1st from 1955-2017 by a Forced Ocean Sea-Ice (FOSI) reconstruction simulation. 


I use the FOSI output as a reference for potential predictability of the CESM-DPLE. I load in ERSST observations for a skill assessment of CESM-DPLE. Lastly, I use the CESM Large Ensemble (CESM-LE) as the uninitialized counterpart of the CESM-DPLE. This was generated by perturbing the CESM atmospheric temperature in 1920 slightly to generate an ensemble that diverges due to the chaotic nature of the climate system.

**References**

Details on the CESM-DPLE experimental setup:

1. Yeager, S. G., et al. "Predicting near-term changes in the Earth System: A large ensemble of initialized decadal prediction simulations using the Community Earth System Model." Bulletin of the American Meteorological Society 99.9 (2018): 1867-1886.

Details on the ERSST observations:

2. https://www.ncdc.noaa.gov/data-access/marineocean-data/extended-reconstructed-sea-surface-temperature-ersst-v5

Details on the CESM Large Ensemble:

3. Kay, J. E., et al. "The Community Earth System Model (CESM) large ensemble project: A community resource for studying climate change in the presence of internal climate variability." Bulletin of the American Meteorological Society 96.8 (2015): 1333-1349.

Here we load the sample data included in `climpred`.

In [ ]:
def _load_dple():
    sst = load_dataset('CESM-DP-SST')
    sss = load_dataset('CESM-DP-SSS')
    dple = xr.merge([sst, sss])
    dple = dple.sel(init=slice(1955, 2015))
    # detrend
    dple['SST'] = cp.stats.rm_trend(dple['SST'], dim='init')
    dple['SSS'] = cp.stats.rm_trend(dple['SSS'], dim='init')
    return dple

def _load_fosi():
    sst = load_dataset('FOSI-SST')
    sss = load_dataset('FOSI-SSS')
    fosi = xr.merge([sst, sss])
    fosi = fosi.sel(time=slice(1955, 2015))
    # move to anomaly space
    fosi = fosi - fosi.sel(time=slice(1964, 2014)).mean('time')
    # detrend
    fosi['SST'] = cp.stats.rm_trend(fosi['SST'])
    fosi['SSS'] = cp.stats.rm_trend(fosi['SSS'])
    return fosi

def _load_data():
    data = load_dataset('ERSST')['SST']
    # move to anomaly space
    data = data - data.sel(time=slice(1964, 2014)).mean('time')
    # detrend
    data = cp.stats.rm_trend(data)
    data.name = 'SST'
    return data

def _load_cesmLE():
    lens = load_dataset('CESM-LE')
    # Take ensemble mean for this demo
    lens = lens.mean('member')
    return lens

dple = _load_dple()
fosi = _load_fosi()
data = _load_data().to_dataset()
cesmLE = _load_cesmLE()

The `HindcastEnsemble` object is created just by inputting the prediction ensemble output. Other reference objects can be added after it is generated.

In [ ]:
dp = cp.HindcastEnsemble(dple)
print(dp)

For a `HindcastEnsemble` object, you can add multiple references. Here, we will add the reconstruction (FOSI) and observations (ERSST). Multiple checks go into play here under the hood, ensuring that dimensions are named according to our rules and that references match all initialized dimensions (except for time).

We can also have multiple variables per reference and decadal prediction object.

In [ ]:
dp.add_reference(fosi, 'FOSI')
dp.add_reference(data, 'ERSST')
print(dp)

Lastly, we can add an uninitialized companion run. **Note that individual products can differ in the variables they have**.

In [ ]:
dp.add_uninitialized(cesmLE)
print(dp)

Now we can apply our functions to our `HindcastEnsemble` object. You can call `compute_metric` in two different ways:

1. Declare a single reference you want to compare the initialized ensemble to (this returns a single `xr.Dataset` with results for that comparison)
1. Run `compute_metric` with no reference argument. It will then make the comparison across all available references and variables. This returns a dictionary with results for each reference.

In [ ]:
# This computes the anomaly correlation coefficient (ACC) between the initialized
# ensemble and FOSI (the reconstruction). The default metric is 'pearson r', so
# we don't need to declare that keyword.
skill = dp.compute_metric('FOSI')
print(skill)

In [ ]:
# Comparison with FOSI, but now with mean absolute error.
# Also declare that we want to compare each individual forecast to
# the reconstruction.
dp.compute_metric('FOSI', metric='mae', comparison='m2r')

In [ ]:
# Now we can go with option (2). Computing skill across all references.
# Automatically computes for all references a pearsonr ensemble mean.
skill = dp.compute_metric()
print(skill)

In [ ]:
print(skill['FOSI'])

In [ ]:
print(skill['ERSST'])

We can also compute a persistence forecast. This automatically only computes persistence for the references.

In [ ]:
# Persistence for both FOSI and data, but only out to 8 lags.
persist = dp.compute_persistence(nlags=8)

In [ ]:
print(persist['FOSI'])

In [ ]:
print(persist['ERSST'])

Lastly, we can compare our uninitialized ensemble to the references to see how much better the initialized ensemble does.

In [ ]:
uninit = dp.compute_uninitialized('FOSI')
print(uninit)

### Plots from our Results

Future versions of `climpred` might have a more advanced plotting method. But since this is built on `xarray`, we can use their native plotting commands. This demo uses `proplot`, an awesome `matplotlib` wrapper (https://github.com/lukelbd/proplot).

Here we plot results for global surface SST:

1. potential predictability (DPLE vs. FOSI)
1. skill (DPLE vs. ERSST)
1. uninitialized reference (CESM-LE vs. FOSI)
1. persistence forecast (FOSI persistence)

In [ ]:
f, ax = plt.subplots(figsize=(7,2))

# Automatically computes pearson_r for the ensemble mean DPLE.
skill = dp.compute_metric()
persistence = dp.compute_persistence('FOSI')
uninit = dp.compute_uninitialized('FOSI')

# Show dashed line for reference
uninit = xr.concat([uninit]*10, 'lead')
uninit['lead'] = persistence['lead']

# Plot results
p1, = ax.plot(skill['FOSI'].lead, skill['FOSI'].SST,
        'o', markersize=7, label='initialized predictability',
        color='r')
p2, = ax.plot(skill['ERSST'].lead, skill['ERSST'].SST,
        'o', markersize=7, label='initialized skill',
        color='b')
p3, = ax.plot(persistence.lead, persistence.SST,
        'o', markersize=7, label='persistence forecast', color='gray')
p4, = ax.plot(uninit.lead[0], uninit.SST[0], 'o', markersize=7, color='y',
        label='uninitialized forecast')
ax.plot(uninit.lead, uninit.SST, '--', color='y', linewidth=1)

ax.set(ylim=[-0.5, 1], xlim=[0.5, 10.5],
       xlabel='lead year', ylabel='anomaly correlation \n coefficient',
       xticks=np.arange(1, 11))
ax.set_title('Detrended Global SST Forecasts', fontsize=12)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.setp(plt.gca().get_legend().get_texts(), fontsize=10)
plt.show()

## Perfect Model Ensemble

This is a brief demo for the functionality of a `PerfectModelEnsemble` object. This has limited development thus far and will be expanded in future PRs.

Here, I use the MPI Perfect Model decadal prediction ensemble. More details on this can be found in the `perfect-model_predictability` notebook.

In [ ]:
ds = load_dataset('MPI-PM-DP-1D')
control = load_dataset('MPI-control-1D')

# get working with just one variable.
area = 'global'
period = 'ym'
ds = ds.sel(area=area, period=period)
control = control.sel(area=area, period=period)

Similar to the `HindcastEnsemble`, you create your `PerfectModelEnsemble` object by adding the prediction model ensemble output. You can then run `add_control()` to add the complementary control run. Here, we add SST ('tos'), SSS ('sos'), and the Atlantic Multidecadal Oscillation index (AMO).

In [ ]:
pm = cp.PerfectModelEnsemble(ds)
pm.add_control(control)
print(pm)

Just as with the `HindcastEnsemble`, we can compare the initialized prediction ensemble to our control run and compute persistence forecasts for the control run.

In [ ]:
# The default here is to compute the pearson r correlation with the 'm2e'
# comparison flag, which generates a supervector to compare all members to
# the ensemble mean.
result = pm.compute_metric(comparison='m2e')
print(result)

In [ ]:
# Here, we compute a simple persistence forecast for all variables.
persist = pm.compute_persistence()
print(persist)

In [ ]:
f, ax = plt.subplots(figsize=(7,2))
p1, = ax.plot(result.lead, result.tos, 'o', color='r', markersize=7,
        label='initialized forecast')
p2, = ax.plot(persist.lead, persist.tos, 'o', color='gray', markersize=7,
        label='persistence forecast')
ax.set(ylim=[-0.75, 1], xlim=[0.5, 20.5],
       ylabel='anomaly correlation \n coefficient',
       xlabel='lead year', xticks=np.arange(1, 21, 2))
ax.set_title('Perfect Model SST Forecasts', fontsize=12)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.setp(plt.gca().get_legend().get_texts(), fontsize=10)
plt.show()

We can reproduce an exercise from the `perfect model` demo notebook here. Now, we use different comparison methods.

In [ ]:
var = 'tos'
for c in ['e2c', 'm2c', 'm2e', 'm2m']:
    pm.compute_metric(comparison=c)[var].plot(label=c)
pm.compute_persistence(nlags=20)[var].plot(label='persistence', ls=':')
plt.ylabel('ACC')
plt.xticks(np.arange(1,21))
plt.legend()
plt.title('Different forecast-reference comparisons for pearson_r' +
          '\n lead to systematically different magnitude of skill score')